# Sample for Kubeflow PyTorchJob SDK

This is a sample for Kubeflow PyTorchJob SDK `kubeflow-pytorchjob`.

The notebook shows how to use Kubeflow PyTorchJob SDK to create, get, wait, check and delete PyTorchJob.

In [1]:
from kubernetes.client import V1PodTemplateSpec
from kubernetes.client import V1ObjectMeta
from kubernetes.client import V1PodSpec
from kubernetes.client import V1Container

from kubeflow.training.utils import utils
from kubeflow.training import V1ReplicaSpec
from kubeflow.training import V1PyTorchJob
from kubeflow.training import V1PyTorchJobSpec
from kubeflow.training import V1RunPolicy
from kubeflow.training import PyTorchJobClient

Define namespace where pytorchjob needs to be created to. If not specified, below function defines namespace to the current one where SDK is running in the cluster, otherwise it will deploy to default namespace.

In [2]:
namespace = utils.get_default_target_namespace()

### Define PyTorchJob

The demo only creates a worker of PyTorchJob to run mnist sample.

In [3]:
container = V1Container(
    name="pytorch",
    image="gcr.io/kubeflow-ci/pytorch-dist-mnist-test:v1.0",
    args=["--backend","gloo"]
)

master = V1ReplicaSpec(
    replicas=1,
    restart_policy="OnFailure",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

worker = V1ReplicaSpec(
    replicas=1,
    restart_policy="OnFailure",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

pytorchjob = V1PyTorchJob(
    api_version="kubeflow.org/v1",
    kind="PyTorchJob",
    metadata=V1ObjectMeta(name="pytorch-dist-mnist-gloo",namespace=namespace),
    spec=V1PyTorchJobSpec(
        run_policy=V1RunPolicy(clean_pod_policy="None"),
        pytorch_replica_specs={"Master": master,
                               "Worker": worker}
    )
)

### Create PyTorchJob

In [4]:
pytorchjob_client = PyTorchJobClient()
pytorchjob_client.create(pytorchjob)

{'apiVersion': 'kubeflow.org/v1',
 'kind': 'PyTorchJob',
 'metadata': {'creationTimestamp': '2022-07-07T09:59:54Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'kubeflow.org/v1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:spec': {'.': {},
      'f:pytorchReplicaSpecs': {'.': {},
       'f:Master': {'.': {},
        'f:replicas': {},
        'f:restartPolicy': {},
        'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}}}},
       'f:Worker': {'.': {},
        'f:replicas': {},
        'f:restartPolicy': {},
        'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}}}}},
      'f:runPolicy': {'.': {}, 'f:cleanPodPolicy': {}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2022-07-07T09:59:54Z'}],
  'name': 'pytorch-dist-mnist-gloo',
  'namespace': 'default',
  'resourceVersion': '31116395',
  'uid': 'ec32aedf-66ca-4d47-9f3f-ae4f344edb42'},
 'spec': {'pytorchReplicaSpecs': {'Master': {'replicas': 1,
    'restartPol

### Get the created PyTorchJob 

In [5]:
pytorchjob_client.get('pytorch-dist-mnist-gloo')

{'apiVersion': 'kubeflow.org/v1',
 'kind': 'PyTorchJob',
 'metadata': {'creationTimestamp': '2022-07-07T09:59:54Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'kubeflow.org/v1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:spec': {'.': {},
      'f:pytorchReplicaSpecs': {'.': {},
       'f:Master': {'.': {},
        'f:replicas': {},
        'f:restartPolicy': {},
        'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}}}},
       'f:Worker': {'.': {},
        'f:replicas': {},
        'f:restartPolicy': {},
        'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}}}}},
      'f:runPolicy': {'.': {}, 'f:cleanPodPolicy': {}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2022-07-07T09:59:54Z'}],
  'name': 'pytorch-dist-mnist-gloo',
  'namespace': 'default',
  'resourceVersion': '31116395',
  'uid': 'ec32aedf-66ca-4d47-9f3f-ae4f344edb42'},
 'spec': {'pytorchReplicaSpecs': {'Master': {'replicas': 1,
    'restartPol

### Get the PyTorchJob status, check if the PyTorchJob has been started.

In [6]:
pytorchjob_client.get_job_status('pytorch-dist-mnist-gloo', namespace=namespace)

'Running'

### Wait for the specified PyTorchJob to finish

In [7]:
pytorchjob_client.wait_for_job('pytorch-dist-mnist-gloo', namespace=namespace, watch=True)

NAME                           STATE                TIME                          
pytorch-dist-mnist-gloo        Running              2022-07-07T09:59:54Z          
pytorch-dist-mnist-gloo        Running              2022-07-07T09:59:54Z          
pytorch-dist-mnist-gloo        Running              2022-07-07T09:59:54Z          
pytorch-dist-mnist-gloo        Succeeded            2022-07-07T10:02:52Z          


### Check if the PyTorchJob succeeded

In [8]:
pytorchjob_client.is_job_succeeded('pytorch-dist-mnist-gloo', namespace=namespace)

True

### Get the PyTorchJob training logs.

In [9]:
pytorchjob_client.get_logs('pytorch-dist-mnist-gloo', namespace=namespace)

The logs of Pod pytorch-dist-mnist-gloo-master-0:
 Using CUDA
Using distributed PyTorch with gloo backend
Processing...
Done!
Train Epoch: 1 [0/60000 (0%)]	loss=2.3000
Train Epoch: 1 [640/60000 (1%)]	loss=2.2135
Train Epoch: 1 [1280/60000 (2%)]	loss=2.1705
Train Epoch: 1 [1920/60000 (3%)]	loss=2.0767
Train Epoch: 1 [2560/60000 (4%)]	loss=1.8681
Train Epoch: 1 [3200/60000 (5%)]	loss=1.4142
Train Epoch: 1 [3840/60000 (6%)]	loss=1.0009
Train Epoch: 1 [4480/60000 (7%)]	loss=0.7757
Train Epoch: 1 [5120/60000 (9%)]	loss=0.4600
Train Epoch: 1 [5760/60000 (10%)]	loss=0.4863
Train Epoch: 1 [6400/60000 (11%)]	loss=0.4378
Train Epoch: 1 [7040/60000 (12%)]	loss=0.4104
Train Epoch: 1 [7680/60000 (13%)]	loss=0.4624
Train Epoch: 1 [8320/60000 (14%)]	loss=0.4293
Train Epoch: 1 [8960/60000 (15%)]	loss=0.3989
Train Epoch: 1 [9600/60000 (16%)]	loss=0.3864
Train Epoch: 1 [10240/60000 (17%)]	loss=0.2982
Train Epoch: 1 [10880/60000 (18%)]	loss=0.5041
Train Epoch: 1 [11520/60000 (19%)]	loss=0.5230
Train Epoc

### Delete the PyTorchJob

In [10]:
pytorchjob_client.delete('pytorch-dist-mnist-gloo')

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'pytorch-dist-mnist-gloo',
  'group': 'kubeflow.org',
  'kind': 'pytorchjobs',
  'uid': 'ec32aedf-66ca-4d47-9f3f-ae4f344edb42'}}